In [9]:
def stumpClassify(dataMatrix, dimen, threshVal, threshIneq) :
    retArray = ones((shape(dataMatrix)[0],1))
    
    if threshIneq == 'lt' :
        retArray[dataMatrix[:, dimen] <= threshVal]  = -1.0
    else :
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    return retArray


In [13]:
def buildStump(dataArr, classLabels,D) : 
    dataMatrix = mat(dataArr); labelMat = mat(classLabels).T
    m, n = shape(dataMatrix)
    numSteps = 10.0 ; bestStump = {}; bestClasEst = mat(zeros((m,1)))
    minError = inf
    for i in range(n) : 
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax - rangeMin)/numSteps
        for j in range(-1, int(numSteps)+1) : 
            for inequal in ['lt','gt'] : 
                threshVal = (rangeMin+float(j)*stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr = mat(ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr
                #print ("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" %(i, threshVal, inequal, weightedError))
                #print(errArr)
                if weightedError < minError:
                    minError = weightedError
                    #print("minError = ",minError)
                    bestClasEst = predictedVals.copy()
                    #print("bestClasEst = ",bestClasEst)
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
                    #print("bestStump =",bestStump)
    return bestStump, minError, bestClasEst
            

In [2]:
from numpy import *
def loadSimpData():
    datMat = matrix([[ 1. , 2.1],
                    [ 2. , 1.1],
                    [ 1.3, 1. ],
                    [ 1. , 1. ],
                    [ 2. , 1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

In [14]:
datMat , classLabels = loadSimpData()

In [15]:
D = mat(ones((5,1))/5)

In [16]:
buildStump(datMat, classLabels, D)

({'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, matrix([[0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

In [21]:
def adaBoostTrainDS(dataArr, classLabels, numIt=40):
    #number of iteration; iteration을 반복함으로써 error를 0에 가깝게 수렴
    weakClassArr = []
    #built for decision stumps
    m = shape(dataArr)[0]
    #받을 데이터 array의 총 데이터의 갯수
    D = mat(ones((m,1))/m)
    #weight vector의 초기값 -- 모두 동일하게
    #increasethe weight of misclassified data + decrease the weight of the properly classified data
    # m으로 나눠주는 이유 ; D는 확률분포이기 때문에 D값들의 sum을 한 것이 1이 되어야함
    aggClassEst = mat(zeros((m,1)))
    # predict한 class matrix  * alpha
    for i in range(numIt) :
        bestStump, error, classEst = buildStump(dataArr, classLabels,D)
        # error : the lowest error returned, classEst : estimated classes for iteration D
        print ("D:",D.T)
        #######  after each classifier is trained , the classifier's weight is calcuated 
       # based on its accuracy. More accurate classifiers are given more weight
        alpha = float(0.5*log((1.0-error)/max(error,1e-16)))
        # alpha : 첫번째 iteration에서 나온 분류기의 output weight 계산하기 based on classifier's error rate
        # error가 커지면 alpha값이 작게설정, max값은 error가 0일때 오류가 나는 것을 방지하기위해 0에 가까운 값설정
        bestStump['alpha'] = alpha
        weakClassArr.append(bestStump)
        #alpha값을 best stump 딕셔너리에 저장, weakClassArr에 append
        print("weakClassArr", weakClassArr)
        print ("classEst : ", classEst.T)
        expon = multiply(-1*alpha*mat(classLabels).T, classEst)
        D = multiply(D, exp(expon))
        D = D/D.sum()
        # weight vector 구하기;  모든 데이터(m개)에 대해서 D를 구함
        # classEst(우리의 예측치) 가 실제 classlabel과 다르면 weight가 더 더해지는 방향. 
        aggClassEst += alpha*classEst
        print("aggClassEst : ", aggClassEst.T)
        aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T, ones((m,1)))
        
        errorRate = aggErrors.sum()/m
        print("total error : ",errorRate, "\n")
        if errorRate == 0.0 : break
            # error =0 이라면 exit loop
    return weakClassArr

In [22]:
weakClassArr = adaBoostTrainDS(datMat, classLabels, 9)

D: [[0.2 0.2 0.2 0.2 0.2]]
weakClassArr [{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3}]
classEst :  [[-1.  1. -1. -1.  1.]]
aggClassEst :  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error :  0.2 

D: [[0.5   0.125 0.125 0.125 0.125]]
weakClassArr [{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0}]
classEst :  [[ 1.  1. -1. -1. -1.]]
aggClassEst :  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error :  0.2 

D: [[0.28571429 0.07142857 0.07142857 0.07142857 0.5       ]]
weakClassArr [{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0}, {'alpha': 0.8958797346140273, 'dim': 0, 'ineq': 'lt', 'thresh': 0.9}]
classEst :  [[1. 1. 1. 1. 1.]]
aggClassEst :  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error :  0.0 

